In [5]:
# from pathlib import Path

# import tensorflow.compat.v1 as tf

from contact_graspnet.orig.contact_graspnet import config_utils
from contact_graspnet.orig.contact_graspnet.data import load_available_input_data
# from contact_grasp_estimator import GraspEstimator
# from visualization_utils import visualize_grasps, show_image

In [ ]:
tf.disable_eager_execution()
physical_devices = tf.config.experimental.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
checkpoint_dir = Path(checkpoint_dir)
if not checkpoint_dir.exists():
    checkpoint_dir = Path(__file__).parent.parent / "checkpoints" / checkpoint_dir
if not checkpoint_dir.exists():
    raise ValueError("Checkpoint directory not found: ", checkpoint_dir)

# load config
global_config = config_utils.load_config(
    checkpoint_dir, batch_size=forward_passes, arg_configs=[]
)

# Build the model
grasp_estimator = GraspEstimator(global_config)
grasp_estimator.build_network()

# Add ops to save and restore all the variables.
saver = tf.train.Saver(save_relative_paths=True)

# Create a session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.allow_soft_placement = True
sess = tf.Session(config=config)

# Load weights
grasp_estimator.load_weights(sess, saver, checkpoint_dir, mode="test")